In [2]:
import pandas as pd
from IPython.display import display, Markdown
import timeit
from tabulate import tabulate
import numpy as np

Перший рівень (спрощені завдання)

In [3]:
headers: list = ["Date", "Time", "Global_active_power", "Global_reactive_power", "Voltage", 
                 "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]

pd_types: dict = {"Date": str, "Time": str, "Global_active_power": float, "Global_reactive_power": float,
              "Voltage": float, "Global_intensity": float, "Sub_metering_1": float, "Sub_metering_2": float,
              "Sub_metering_1": float}

np_types: list[tuple] = [("Date", "U10"), ("Time", "U8"), ("Global_active_power",
              "float64"), ("Global_reactive_power", "float64"), ("Voltage",
              "float64"), ("Global_intensity", "float64"), ("Sub_metering_1",
              "float64"), ("Sub_metering_2", "float64"), ("Sub_metering_3",
              "float64")]

In [4]:
pd_df = pd.read_csv("household_power_consumption.txt", delimiter=';',
                 dtype=pd_types, na_values="?",  header=1, names=headers)

pd_df["Time"] = pd.to_datetime(pd_df["Time"], format="%H:%M:%S").dt.time
pd_df['Date'] = pd.to_datetime(pd_df["Date"], format="%d/%m/%Y")

#Dropping NaN values
# pd_df.dropna(inplace=True)

#Changing NaN values
pd_df.fillna(pd_df.mean(numeric_only=True), inplace=True)



In [5]:
def pd_sort_gap(value: int) -> None:
      display(Markdown(" ### ⚡All entries with Global_active_power higher than 5 kW"))
      display(pd_df[pd_df["Global_active_power"] > value].head(20))

def pd_sort_voltage(value: int) -> None:
      display(Markdown(" ### ⚡All entries with Voltage higher than 235 V"))
      display(pd_df[pd_df["Voltage"] > value].head(20))

def pd_sort_gi(values: tuple) -> None:
      display(Markdown(" ### ⚡All entries with Global_intensity between 19-20 A \
                        and \"Sub_metering_2\" > \"Sub_metering_3\""))
      display(pd_df[(pd_df["Global_intensity"].between(values[0], values[1]))
                     & (pd_df["Sub_metering_2"]> pd_df["Sub_metering_3"])].head(20))
      
def pd_sort_sample() -> None:
      sample_df = pd_df.sample(n=500000, replace=False, random_state=42)
      avg_values = sample_df[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]].mean()
      display(Markdown(f''' ### ⚡ Average values of energy consumption:
            Sub_metering_1: {avg_values["Sub_metering_1"]:.2f}
            Sub_metering_2: {avg_values["Sub_metering_2"]:.2f}
            Sub_metering_3: {avg_values["Sub_metering_3"]:.2f}
            '''))

def pd_multiple_sort(value: int) -> None:
      sorted_df = pd_df[(pd_df["Time"] > pd.to_datetime("18:00:00").time()) & 
                        (pd_df["Global_active_power"] > value)]
      
      sorted_df = sorted_df[(sorted_df["Sub_metering_2"] > sorted_df["Sub_metering_1"]) &
                           (sorted_df["Sub_metering_2"] > sorted_df["Sub_metering_3"])]
      
      half_len = len(sorted_df)//2

      first_half = sorted_df.iloc[:half_len]
      second_half = sorted_df.iloc[half_len:]
      
      final_df = pd.concat([first_half.iloc[::3], second_half.iloc[::4]])
      display(final_df.head(20))


In [6]:
#
a = timeit.timeit(lambda: pd_sort_gap(value=5), number=1)
print(f"Time of execution: {a}")
#
b = timeit.timeit(lambda: pd_sort_voltage(value=235), number=1)
print(f"Time of execution: {b}")

#
c = timeit.timeit(lambda: pd_sort_gi(values=(19, 20)), number=1)
print(f"Time of execution: {c}")

#
d = timeit.timeit(lambda: pd_sort_sample(), number=1)
print(f"Time of execution: {d}")

#
e = timeit.timeit(lambda: pd_multiple_sort(value=6), number=1)
print(f"Time of execution: {e}")

 ### ⚡All entries with Global_active_power higher than 5 kW

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
1,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
10,2006-12-16,17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
11,2006-12-16,17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0
12,2006-12-16,17:37:00,5.268,0.398,232.91,22.6,0.0,2.0,17.0
19,2006-12-16,17:44:00,5.894,0.000,232.69,25.4,0.0,0.0,16.0
20,2006-12-16,17:45:00,7.706,0.000,230.98,33.2,0.0,0.0,17.0
21,2006-12-16,17:46:00,7.026,0.000,232.21,30.6,0.0,0.0,16.0
22,2006-12-16,17:47:00,5.174,0.000,234.19,22.0,0.0,0.0,17.0


Time of execution: 0.1256201000069268


 ### ⚡All entries with Voltage higher than 235 V

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
3,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
4,2006-12-16,17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
5,2006-12-16,17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
6,2006-12-16,17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
13,2006-12-16,17:38:00,4.054,0.422,235.24,17.6,0.0,1.0,17.0
14,2006-12-16,17:39:00,3.384,0.282,237.14,14.2,0.0,0.0,17.0
15,2006-12-16,17:40:00,3.270,0.152,236.73,13.8,0.0,0.0,17.0
16,2006-12-16,17:41:00,3.430,0.156,237.06,14.4,0.0,0.0,17.0
17,2006-12-16,17:42:00,3.266,0.000,237.13,13.8,0.0,0.0,18.0
18,2006-12-16,17:43:00,3.728,0.000,235.84,16.4,0.0,0.0,17.0


...

Time of execution: 0.06391069997334853


 ### ⚡ Average values of energy consumption:
            Sub_metering_1: 1.12
            Sub_metering_2: 1.30
            Sub_metering_3: 6.46
            

Time of execution: 0.4118511000124272


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
40,2006-12-16,18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0
43,2006-12-16,18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0
17493,2006-12-28,20:58:00,6.386,0.374,236.63,27.0,1.0,36.0,17.0
17497,2006-12-28,21:02:00,8.088,0.262,235.50,34.4,1.0,72.0,17.0
17500,2006-12-28,21:05:00,7.230,0.152,235.22,30.6,1.0,73.0,17.0
17503,2006-12-28,21:08:00,7.352,0.000,235.45,31.2,1.0,73.0,17.0
17506,2006-12-28,21:11:00,9.048,0.000,231.48,39.0,34.0,71.0,16.0
17509,2006-12-28,21:14:00,9.118,0.108,231.18,39.4,36.0,72.0,16.0
17512,2006-12-28,21:17:00,7.040,0.130,233.27,30.2,37.0,38.0,17.0
18951,2006-12-29,21:16:00,6.146,0.116,230.53,26.6,0.0,70.0,0.0


Time of execution: 0.17366470000706613


Numpy implementation

In [7]:
np_df = np.genfromtxt("household_power_consumption.txt", 
       missing_values=["?", np.nan], delimiter=';', 
       dtype=np_types, encoding="UTF=8", names=True)



numeric_columns = [header for header in headers if np_df[header].dtype in ['float64', 'int64']]
means = {col: np.nanmean(np_df[col]) for col in numeric_columns}

for col in numeric_columns:
    np_df[col] = np.nan_to_num(np_df[col], nan=means[col])


In [8]:
def np_sort_gap(value: int) -> None:
      display(Markdown(" ### ⚡All entries with Global_active_power higher than 5 kW"))
      print(tabulate(np_df[np_df["Global_active_power"] > value][:20], headers=np_df.dtype.names, tablefmt='rounded_grid'))

def np_sort_voltage(value: int) -> None:
      display(Markdown(" ### ⚡All entries with Voltage higher than 235 V"))
      print(tabulate(np_df[np_df["Voltage"] > value][:20], headers=np_df.dtype.names, tablefmt='rounded_grid'))

def np_sort_gi(values: tuple) -> None:
      display(Markdown(" ### ⚡All entries with Global_intensity between 19-20 A \
                        and \"Sub_metering_2\" > \"Sub_metering_3\""))
      print(tabulate(np_df[(np_df["Global_intensity"] >= values[0]) & 
                   (np_df["Global_intensity"] <= values[1]) & 
                   (np_df["Sub_metering_2"] > np_df["Sub_metering_3"])][:20], headers=np_df.dtype.names, tablefmt='rounded_grid'))
      
def np_sort_sample() -> None:
      np.random.seed(42)
      sample_df = np.random.choice(np_df, size=500000, replace=False)
      means = {col: np.mean(sample_df[col]) for col in ["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]}
      display(Markdown(f''' ### ⚡ Average values of energy consumption:
            Sub_metering_1: {means["Sub_metering_1"]:.2f}
            Sub_metering_2: {means["Sub_metering_2"]:.2f}
            Sub_metering_3: {means["Sub_metering_3"]:.2f}
            '''))

def np_multiple_sort(value: int) -> None:
      sorted_df = np_df[(np_df["Time"] > "18:00:00") & 
                        (np_df["Global_active_power"] > value)]
      
      sorted_df = sorted_df[(sorted_df["Sub_metering_2"] > sorted_df["Sub_metering_1"]) &
                           (sorted_df["Sub_metering_2"] > sorted_df["Sub_metering_3"])]
      
      half_len = len(sorted_df)//2

      first_half = sorted_df[:half_len]
      second_half = sorted_df[half_len:]
      
      final_df = np.concatenate([first_half[::3], second_half[::4]])
      print(tabulate(final_df, headers=final_df.dtype.names, tablefmt='rounded_grid'))

In [9]:
#
f = timeit.timeit(lambda: np_sort_gap(value=5), number=1)
print(f"Time execution: {f}")

#
g = timeit.timeit(lambda: np_sort_voltage(value=235), number=1)
print(f"Time execution: {g}")

#
h = timeit.timeit(lambda: np_sort_gi(values=(19, 20)), number=1)
print(f"Time execution: {h}")

#
i = timeit.timeit(lambda: np_sort_sample(), number=1)
print(f"Time execution: {i}")

#
j = timeit.timeit(lambda: np_multiple_sort(value=6), number=1)
print(f"Time execution: {j}")

 ### ⚡All entries with Global_active_power higher than 5 kW

╭────────────┬──────────┬───────────────────────┬─────────────────────────┬───────────┬────────────────────┬──────────────────┬──────────────────┬──────────────────╮
│ Date       │ Time     │   Global_active_power │   Global_reactive_power │   Voltage │   Global_intensity │   Sub_metering_1 │   Sub_metering_2 │   Sub_metering_3 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 17:25:00 │                 5.36  │                   0.436 │    233.63 │               23   │                0 │                1 │               16 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 17:26:00 │                 5.374 │                   0.498 │    233.29 │               23   │                0 │                2 │               17 │
├───

 ### ⚡All entries with Voltage higher than 235 V

╭────────────┬──────────┬───────────────────────┬─────────────────────────┬───────────┬────────────────────┬──────────────────┬──────────────────┬──────────────────╮
│ Date       │ Time     │   Global_active_power │   Global_reactive_power │   Voltage │   Global_intensity │   Sub_metering_1 │   Sub_metering_2 │   Sub_metering_3 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 17:28:00 │                 3.666 │                   0.528 │    235.68 │               15.8 │                0 │                1 │               17 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 17:29:00 │                 3.52  │                   0.522 │    235.02 │               15   │                0 │                2 │               17 │
├───

 ### ⚡All entries with Global_intensity between 19-20 A                         and "Sub_metering_2" > "Sub_metering_3"

╭────────────┬──────────┬───────────────────────┬─────────────────────────┬───────────┬────────────────────┬──────────────────┬──────────────────┬──────────────────╮
│ Date       │ Time     │   Global_active_power │   Global_reactive_power │   Voltage │   Global_intensity │   Sub_metering_1 │   Sub_metering_2 │   Sub_metering_3 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 18:09:00 │                 4.464 │                   0.136 │    234.66 │               19   │                0 │               37 │               16 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 17/12/2006 │ 01:04:00 │                 4.582 │                   0.258 │    238.08 │               19.6 │                0 │               13 │                0 │
├───

 ### ⚡ Average values of energy consumption:
            Sub_metering_1: 1.12
            Sub_metering_2: 1.30
            Sub_metering_3: 6.47
            

Time execution: 0.6172801000066102
╭────────────┬──────────┬───────────────────────┬─────────────────────────┬───────────┬────────────────────┬──────────────────┬──────────────────┬──────────────────╮
│ Date       │ Time     │   Global_active_power │   Global_reactive_power │   Voltage │   Global_intensity │   Sub_metering_1 │   Sub_metering_2 │   Sub_metering_3 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 18:05:00 │                 6.052 │                   0.192 │    232.93 │               26.2 │                0 │               37 │               17 │
├────────────┼──────────┼───────────────────────┼─────────────────────────┼───────────┼────────────────────┼──────────────────┼──────────────────┼──────────────────┤
│ 16/12/2006 │ 18:08:00 │                 6.308 │                   0.116 │    232.25 │               27   │                0 │        

In [10]:
print("Time using pandas dataframes: ")
print(a, b, c, d, e, sep=" sec\n", end=" sec\n")
print()
print("Time using numpy arrays: ")
print(f, g, h, i, j, sep=" sec\n", end=" sec\n")

Time using pandas dataframes: 
0.1256201000069268 sec
0.2876581000164151 sec
0.06391069997334853 sec
0.4118511000124272 sec
0.17366470000706613 sec

Time using numpy arrays: 
0.06342970000696369 sec
0.33258139999816194 sec
0.13762360002147034 sec
0.6172801000066102 sec
0.15616210000007413 sec


Бачимо, що в цілому час виконання функцій з використанням обох структур приблизно однаковий. 